## Feature Engineering and CV based Winners' Solutions

Try to use MAP@7 as `feval` for xgboost, failed because no way to tell xgboost how to group results by users. **Can only do this after train is finished.**

New in this notebook:
- A hacky implementation of MAP@7 evaluation. 
- This method is suitable when training on one month and validate on another month, since ncodpers is the key in ground truth dictionaries.
- This method **only works if the MAP functions and training codes are in the same notebook**.
- The MAP values shown during training is averaged by the number of customers with new products, so very different from LB. But it gives a sense of if the algorithm is good or not.

To-do: 
- mean encoding of products grouped by combinations of: canal_entrada, segmento, cod_prov
- Time since change and lags for a few non-product features: 
    - segmento
    - ind_actividad_cliente
    - cod_prov
    - canal_entrada
    - indrel_1mes
    - tiprel_1mes


Features:
- before eda_4_29
    - average of products for each (customer, product) pair
    - exponent weighted average of products each (customer, product) pair
    - time since presence of products, distance to the first 1
    - time to the last positive flank (01)
    - time to the last negative flank (10)
    - time to the last 1, to the nearest product purchase
    - time to the first 1, to the first product purchase
    - Trained@2015-06-28, validated@2015-12-28, mlogloss=1.28481
    - Private score: 0.0302054, public score: 0.0298683
- before eda_4_25
    - customer info in the second month
    - products in the first month
    - combination of first and second month `ind_actividad_cliente`
    - combination of first and second month `tiprel_1mes`
    - combination of first month product by using binary number (`target_combine`)
    - encoding `target_combine` with 
        - mean number of new products
        - mean number of customers with new products
        - mean number of customers with each new products
    - Count patterns in the last `max_lag` months
    - Number of month to the last time the customer purchase each product
        - CV@2015-12-28: mlogloss=1.29349
        - Private score: 0.0302475, public score: 0.0299266
- eda_4_25
    - Use all available history data
        - E.g., for 2016-05-28 train data, use all previous months, for 2015-02-28, use 1 lag month. 
        - Need to create test set that use the same amount of previous months for each training data set. 
        - This is from [the second winner's solution](https://www.kaggle.com/c/santander-product-recommendation/discussion/26824), his bold part in paragraph 4.
    - Combine models trained on 2016-05-28 and 2015-06-28:
        - Private score: 0.0304583, public score: 0.0300839
        - This is to catch both seasonality and trend, presented in 2015-06-28 and 2016-05-28, respectively. 
        - This idea is mentioned by many winners, like [11-th winner](https://www.kaggle.com/c/santander-product-recommendation/discussion/26823) and [14-th winner](https://www.kaggle.com/c/santander-product-recommendation/discussion/26808)

- eda_4_27
    - put 2015-06-28 and 2016-05-28 in the same data set, with the same lag=5
        - Private score:0.0303096, public score: 0.0299867
        - Different as [11-th winner's discussion](https://www.kaggle.com/c/santander-product-recommendation/discussion/26823)
            > We tested this by adding 50% of May-16 data to our June model and sure enough, we went from 0.0301 to 0.0303. Then, we built separate models for Jun and May, but the ensemble didn’t work. We weren’t surprised because June data is better for seasonal products, and May data is better for trend products. And vice-versa, June data is bad for trend products and May data is bad for seasonal products. So, they sort of cancelled each other out.

        - But my score is always worse than theirs, maybe this is the reason why we have different observations

# Compare two weights

In [1]:
from santander_helper import *

In [ ]:
param = {'objective': 'multi:softprob', 
         'eta': 0.05, 
         'max_depth': 8, 
         'silent': 1, 
         'num_class': len(target_cols),
         'eval_metric': 'mlogloss',
         'min_child_weight': 1,
         'subsample': 0.7,
         'colsample_bytree': 0.7,
         'seed': 0}
num_rounds = 150
month_train = '2015-06-28'
month_val = '2016-05-28'
weights = (4, 5, 6, 7, 8, 9)
weights = (10, 11, 12, 13, 14, 15)

history, model_dict = cv_month(param, num_rounds, month_train, month_val, n_repeat=7, random_seed=240,
                    lag_train=5, lag_val=5, weight_set=weights, verbose_eval=False)

Start weight index 10
##################################################
Train with weight 10, repetition 0 of 7


In [ ]:
history_prev = pd.read_csv('weight_cv_2_history.csv', header=[0, 1, 2, 3], index_col=0)
history_prev.columns.set_levels(history_prev.columns.levels[0].astype(int), level=0, inplace=True) # convert string to int
history_prev.columns.set_levels(history_prev.columns.levels[1].astype(int), level=1, inplace=True) # convert string to int
val = {}

# load previous result w=exp(1/n-1)
val[1, 'mean'] = history_prev.xs(axis=1, level=[0, 2, 3], key=(9, 'val', 'MAP@7')).apply('mean', axis=1)
val[1, 'std'] = history_prev.xs(axis=1, level=[0, 2, 3], key=(9, 'val', 'MAP@7')).apply('std', axis=1)

# process current results
for idx, metric in itertools.product(weights, ['mean', 'std']):
    val[(idx, metric)] = history.xs(axis=1, level=[0, 2, 3], key=(idx, 'val', 'MAP@7')).apply(metric, axis=1)

In [ ]:
plt.figure(figsize=(16, 9))

plt.plot(val[1, 'mean'], label='weight=exp(1/n-1)')
plt.fill_between(history_prev.index, val[1, 'mean']-val[1, 'std'], val[1, 'mean']+val[1, 'std'], alpha=0.3)

nd = {10: 0.6, 11: 0.5, 12: 0.4, 13: 0.3, 14: 0.2, 15: 0.1}
for idx in weights:
    plt.plot(val[idx, 'mean'], label='weight=exp({}/n-{})'.format(nd[idx], nd[idx]))
    plt.fill_between(history.index, val[idx, 'mean']-val[idx, 'std'], val[idx, 'mean']+val[idx, 'std'], alpha=0.3)

plt.grid()
# plt.xlim(50, 100)
# plt.ylim(0.85, 0.86)
plt.legend()

In [ ]:
history.to_csv('weight_cv_3_history.csv')

In [ ]:
save_pickle('weight_cv_3_models.pickle', model_dict)

In [ ]:
history